# Set-Up

## Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import transformers

from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DataCollatorWithPadding
from torch.utils.data import DataLoader
from torch.optim import AdamW

## Constants

In [ ]:
NUM_AUTHORS = 5
RANDOM_STATE = 42

In [ ]:
transformers.set_seed(RANDOM_STATE)

# Fine-Tuning RoBERTa

In [ ]:
full_dataset = load_dataset("csv", data_files =
 {"train": "blog_train.csv", "validation": "blog_val.csv", "test": "blog_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Load Model & Tokenizer

In [ ]:
pretrained_checkpoint = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(pretrained_checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
baseline_model = RobertaForSequenceClassification.from_pretrained(pretrained_checkpoint, num_labels = NUM_AUTHORS)
finetuned_model = RobertaForSequenceClassification.from_pretrained(pretrained_checkpoint, num_labels = NUM_AUTHORS)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
baseline_model = baseline_model.to(device)
finetuned_model = finetuned_model.to(device)
print(device)

cuda


## Apply Tokenization on Dataset

In [ ]:
def tokenize(examples):
    inputs = examples["text"]
    return tokenizer(inputs, max_length = 256, truncation = True, padding="max_length")

In [ ]:
tokenized_ds = full_dataset.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/10656 [00:00<?, ? examples/s]

Map:   0%|          | 0/1332 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds.class_encode_column("labels")
tokenized_ds.set_format("torch")

Stringifying the column:   0%|          | 0/10656 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10656 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/1332 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1332 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/1333 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1333 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds['train'].features

{'text': Value('string'),
 'labels': Value('int64'),
 'input_ids': List(Value('int32')),
 'attention_mask': List(Value('int8'))}

## Training

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_score(labels, preds)

    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(labels, preds, average='micro')

    return {
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1
    }

In [ ]:
num_train_epochs = 5

In [ ]:
training_args_fine_tune = TrainingArguments(
    output_dir = "./fine-tune-roberta-blog-corpus",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_dir = "./fine-tune-roberta-blog-corpus-logs",
    learning_rate = 5e-5,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    push_to_hub = False,
    report_to = "none",
)

In [ ]:
trainer_fine_tune = Trainer(
    model = finetuned_model,
    args = training_args_fine_tune,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    processing_class = tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

for param in baseline_model.roberta.parameters():
    param.requires_grad = False

trainer_baseline = Trainer(
    model=baseline_model,
    args = TrainingArguments(
        report_to = "none",
    ),
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

/tmp/ipython-input-2400842937.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_baseline = Trainer(


In [ ]:
trainer_fine_tune.train()

Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1,Micro Precision,Micro Recall,Micro F1
1,0.855900,0.633811,0.660661,0.465967,0.590158,0.499491,0.660661,0.660661,0.660661
2,0.641300,0.639128,0.623874,0.478144,0.589848,0.486153,0.623874,0.623874,0.623874
3,0.610800,0.619449,0.622372,0.540468,0.586602,0.491934,0.622372,0.622372,0.622372
4,0.580800,0.617837,0.622372,0.561504,0.587205,0.491136,0.622372,0.622372,0.622372
5,0.587200,0.590378,0.616366,0.533465,0.579408,0.495385,0.616366,0.616366,0.616366


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=3330, training_loss=0.6372269318268464, metrics={'train_runtime': 2590.9252, 'train_samples_per_second': 20.564, 'train_steps_per_second': 1.285, 'total_flos': 7009467315240960.0, 'train_loss': 0.6372269318268464, 'epoch': 5.0})

## Evaluation

In [ ]:
pred_baseline = trainer_baseline.predict(tokenized_ds['test'])

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
pred_baseline.metrics

{'test_loss': 1.5948657989501953,
 'test_model_preparation_time': 0.0028,
 'test_accuracy': 0.3143285821455364,
 'test_macro_precision': 0.06291291291291291,
 'test_macro_recall': 0.19952380952380952,
 'test_macro_f1': 0.095662100456621,
 'test_micro_precision': 0.3143285821455364,
 'test_micro_recall': 0.3143285821455364,
 'test_micro_f1': 0.3143285821455364,
 'test_runtime': 18.426,
 'test_samples_per_second': 72.344,
 'test_steps_per_second': 9.063}

In [ ]:
pred_fine_tune = trainer_fine_tune.predict(tokenized_ds['test'])

In [ ]:
pred_fine_tune.metrics

{'test_loss': 0.5959749221801758,
 'test_accuracy': 0.6466616654163541,
 'test_macro_precision': 0.5830308193245495,
 'test_macro_recall': 0.5804258017465663,
 'test_macro_f1': 0.51808264039201,
 'test_micro_precision': 0.6466616654163541,
 'test_micro_recall': 0.6466616654163541,
 'test_micro_f1': 0.6466616654163541,
 'test_runtime': 17.0597,
 'test_samples_per_second': 78.138,
 'test_steps_per_second': 4.924}